In [ ]:
#default_exp database

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, select

import pandas as pd

In [ ]:
#export
def get_db_engine(username, password, ip_adress, model_name, rdbms="mysql"):
    return create_engine("%s://%s:%s@%s/%s" % (rdbms, username, password, ip_adress, model_name),echo = False)

In [ ]:
# Getting the DB engine with credentials.
import getpass
user = getpass.getpass(prompt='Username: ')
psswd = getpass.getpass(prompt='Password: ')
db_adress = getpass.getpass(prompt='DB IP: ')
engine = get_db_engine(user, psswd, db_adress, "asari", "mysql")

Username: ········
Password: ········
DB IP: ········


In [ ]:
#export
def get_record_essentials(record_id, engine):
    q_select_record = "SELECT * FROM Record WHERE id = %d" % record_id
    q_select_cell = "SELECT * FROM Cell WHERE record_id = %d" % record_id
    
    df_record = pd.read_sql_query(q_select_record, engine)
    df_cell = pd.read_sql_query(q_select_cell, engine)
    
    experiment_id = df_record["experiment_id"][0]
    q_select_experiment = "SELECT * FROM Experiment WHERE id = %d" % experiment_id
    df_experiment = pd.read_sql_query(q_select_experiment, engine)
    
    mouse_id = df_experiment["mouse_id"][0]
    q_select_mouse = "SELECT * FROM Mouse WHERE id = %d" % mouse_id
    df_mouse = pd.read_sql_query(q_select_mouse, engine)
    
    tool_id = df_record["tool_id"][0]
    q_select_tool = "SELECT * FROM Tool WHERE id = %d" % tool_id
    df_tool = pd.read_sql_query(q_select_tool, engine)
    
    q_select_map = "SELECT * FROM Map WHERE tool_id = %d" % tool_id
    df_map =  pd.read_sql_query(q_select_map, engine)
    
    res_dict = {"record": df_record, "cell": df_cell, 
                "experiment": df_experiment, "mouse": df_mouse,
                "tool": df_tool, "map": df_map}
    return res_dict

In [ ]:
df_dict = get_record_essentials(100, engine)

In [ ]:
df_dict["mouse"]

,id,sex,date_of_birth,weight,skull_scale,strain,cage,ear_tag,comment
0,93,F,2019-05-28,None,None,C57BL/6JMdo,None,None,M4 ephy tom


In [ ]:
def get_stim_params(stim_hashes, engine):   
    #Writting the query speed up the function rather than querying all individual tables
    # and filtering them all
    query = """SELECT Stim.name AS stim_name, description, barcode, stim_comment, stimulus_id,
                screen_id, hash, date AS date_compiled, compiled_comment, compiled_id, parameter_id,
                Parameter.name as param_name, value as param_value
                FROM (SELECT Compiled.id as comp_id, name, description, barcode, Stimulus.comment AS stim_comment, stimulus_id, screen_id, hash, date, Compiled.comment AS compiled_comment FROM Stimulus 
                LEFT JOIN Compiled ON stimulus_id=Stimulus.id WHERE hash IN """+str(tuple(stim_hashes))+""") AS Stim
                LEFT JOIN Compiled_Parameter ON compiled_id = comp_id
                LEFT JOIN Parameter ON parameter_id = Parameter.id"""
    df_params = pd.read_sql_query(query, engine)
    return df_params

In [ ]:
#Example hashes
stim_hashes = ["14e4cb9b9bfea52be550a4deb5a78340", "0a35a30e4016ddbf65f0c8e854e7ab72"]
get_stim_params(stim_hashes, engine)

,stim_name,description,barcode,stim_comment,stimulus_id,screen_id,hash,date_compiled,compiled_comment,compiled_id,parameter_id,param_name,param_value
0,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,14,last_frame_id,-12
1,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,18,red_sync,cycle
2,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,26,first_frame_id,28
3,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,30,n_repeat,10
4,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,45,chirp_duration,10
5,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,74,contrast_frequency,1.5
6,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,77,tSteadyOFF_s,2.0
7,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,78,tSteadyON_s,2.0
8,chirp_am,Chirp modulated on the amplitude,132131,None,5,4,14e4cb9b9bfea52be550a4deb5a78340,2019-07-29,None,127,79,tSteadyMID_s,1.0
9,flickering_bars_pr,Random presentation black/white vertical bars ...,142111,None,13,1,0a35a30e4016ddbf65f0c8e854e7ab72,2019-09-02,None,143,6,red_sync,on-off


In [ ]:
#hide
#Some examples for SQLAlchemy for later

ins = students.insert()
ins = users.insert().values(name = 'Karan')
ins.compile().params

conn = engine.connect()
ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')
result = conn.execute(ins)
result.inserted_primary_key

conn.execute(students.insert(), [
   {'name':'Rajiv', 'lastname' : 'Khanna'},
   {'name':'Komal','lastname' : 'Bhandari'},
   {'name':'Abdul','lastname' : 'Sattar'},
   {'name':'Priya','lastname' : 'Rajhans'},
])

s = students.select()
result = conn.execute(s)
row = result.fetchone()
for row in result:
    print(row)

In [ ]:
from nbdev.export import *
notebook2script()